In [1]:
import os
import math
import pandas as pd
from datetime import datetime

def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return f"{s} {size_name[i]}"

def get_file_info(directory):
    if not os.path.exists(directory):
        print(f"Erro: O diretório '{directory}' não existe.")
        return None

    print(f"Analisando o diretório: {directory}")
    
    # Lista de dicionários para armazenar as informações dos arquivos
    files_info = []

    # Itera sobre os arquivos no diretório
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            file_stats = os.stat(file_path)

            # Adiciona as informações do arquivo ao dicionário
            files_info.append({
                'Name': file,
                'last_mod_date': datetime.fromtimestamp(file_stats.st_mtime).strftime('%Y-%m-%d %H:%M:%S'),
                'Size': convert_size(file_stats.st_size)
            })

    # Cria um DataFrame do pandas com as informações dos arquivos
    df = pd.DataFrame(files_info)
    df = df.sort_values('last_mod_date')
    

    return df

# Diretório que você deseja analisar
directory = 'data/tratados/2024-03-31'  # ou o caminho do diretório que você deseja analisar

# Verifica se o diretório existe e gera o DataFrame com as informações dos arquivos
file_info_df = get_file_info(directory)

if file_info_df is not None:
    # Exibe o DataFrame
    print(file_info_df)

else:
    print("Não foi possível gerar o DataFrame devido a erros.")


Erro: O diretório 'data/tratados/2024-03-31' não existe.
Não foi possível gerar o DataFrame devido a erros.


In [2]:

# Diretório que você deseja analisar
directory = 'data/temp_dir_202407'  # ou o caminho do diretório que você deseja analisar

# Verifica se o diretório existe e gera o DataFrame com as informações dos arquivos
file_info_df = get_file_info(directory)

if file_info_df is not None:
    # Exibe o DataFrame
    print(file_info_df)

else:
    print("Não foi possível gerar o DataFrame devido a erros.")


Analisando o diretório: data/temp_dir_202407
                                 Name        last_mod_date       Size
5279     processed_group_9999.parquet  2024-09-02 21:33:04    1.76 MB
1999  processed_group_1100015.parquet  2024-09-02 21:33:04    55.5 KB
3331  processed_group_1100023.parquet  2024-09-02 21:33:08  416.02 KB
2329  processed_group_1100031.parquet  2024-09-02 21:33:08    15.3 KB
3445  processed_group_1100049.parquet  2024-09-02 21:33:10  372.98 KB
...                               ...                  ...        ...
5315  processed_group_5221908.parquet  2024-09-02 23:17:17   13.41 KB
4791  processed_group_5222005.parquet  2024-09-02 23:17:17   53.33 KB
5529  processed_group_5222203.parquet  2024-09-02 23:17:17   18.61 KB
3516  processed_group_5222302.parquet  2024-09-02 23:17:18   15.45 KB
3093  processed_group_5300108.parquet  2024-09-02 23:18:44    8.87 MB

[5571 rows x 3 columns]


In [31]:

# Diretório que você deseja analisar
directory = 'data/intermediary'  # ou o caminho do diretório que você deseja analisar

# Verifica se o diretório existe e gera o DataFrame com as informações dos arquivos
df = get_file_info(directory)


Analisando o diretório: data/intermediary


In [12]:
df

,Name,last_mod_date,Size
1,part_2.parquet,2024-07-26 19:35:52,10.91 KB
0,part_1.parquet,2024-07-26 19:38:33,10.91 KB


In [10]:
def find_saved_munis():
    output_dir = r'data/intermediary'
    df = pd.concat([pd.read_parquet(f"{output_dir}/{f}", columns=['cod_muni']).assign(Name = f) for f in os.listdir(output_dir)])
    
    return df

find = find_saved_munis()

ArrowInvalid: No match for FieldRef.Name(cod_muni) in 

In [8]:
find = find.groupby('Name').agg({'cod_muni': ['nunique', 'count']}).reset_index()
find.columns = ['Name', 'nunique', 'count']

NameError: name 'find' is not defined

In [9]:
print(f'Total lines processed: {find["count"].sum()}')

NameError: name 'find' is not defined

In [151]:
df = df.merge(find, on='Name')

In [152]:
df['lag_mod'] = df['last_mod_date'].shift()
df['minutes'] = ((pd.to_datetime(df['last_mod_date']) -  pd.to_datetime(df['lag_mod'])).dt.seconds/60).round(3)#.shift(-1)

In [153]:
df

,Name,last_mod_date,Size,nunique,count,lag_mod,minutes
0,part_1.parquet,2024-07-25 15:58:19,288.54 KB,1,3986,NaN,NaN
1,part_2.parquet,2024-07-25 15:58:40,88.15 MB,1,1337019,2024-07-25 15:58:19,0.350
2,part_3.parquet,2024-07-25 15:59:34,29.37 MB,10,439403,2024-07-25 15:58:40,0.900
3,part_4.parquet,2024-07-25 16:00:04,29.1 MB,5,433929,2024-07-25 15:59:34,0.500
4,part_5.parquet,2024-07-25 16:00:43,32.94 MB,7,485452,2024-07-25 16:00:04,0.650
5,part_6.parquet,2024-07-25 16:01:38,32.63 MB,11,493060,2024-07-25 16:00:43,0.917
6,part_7.parquet,2024-07-25 16:02:31,26.18 MB,11,398087,2024-07-25 16:01:38,0.883
7,part_8.parquet,2024-07-25 16:03:05,17.69 MB,7,261973,2024-07-25 16:02:31,0.567
8,part_9.parquet,2024-07-25 16:03:17,43.49 MB,1,651239,2024-07-25 16:03:05,0.200
9,part_10.parquet,2024-07-25 16:04:25,29.73 MB,15,443061,2024-07-25 16:03:17,1.133


In [13]:
output_dir = r'data/tratados'
df = pd.read_parquet(f"{output_dir}/part_7.parquet")

In [14]:
df

,group_id,group_id_index,nome_muni,uf,merchant_market_hierarchy_id,nome_master,subs_asterisk,documento_final,source_document,merchant_tax_id,...,has_mp,has_mp_subs,has_outros,has_pagseguro,has_sumup,has_ton,has_not_ton,qtd_mmhid_tax_id,grouped_tax_id,qtd_nomes
0,HINODE|Ipojuca PE - 575176432,1,Ipojuca,PE,<NA>,HINODE,SumUp,16668076000120,Places,None,...,True,True,True,True,True,True,True,21378.0,227804,1
1,HINODE|Ipojuca PE - 575176432,1,Ipojuca,PE,575176432,HINODE,Outros,16668076000120,Places,16668076000120,...,True,True,True,True,True,True,True,21378.0,227804,1
2,HINODE|Ipojuca PE - 614429530,2,Ipojuca,PE,614429530,HINODE,Outros,None,Sem Documento,None,...,True,True,True,True,True,True,True,NaN,227804,1
3,HINODE|Ipojuca PE - 575176432,1,Ipojuca,PE,<NA>,HINODE,CloudWalk,16668076000120,Places,None,...,True,True,True,True,True,True,True,21378.0,227804,1
4,TAYNARADELACQUAWALD|Arapongas PR - 784064469,3,Arapongas,PR,<NA>,TAYNARADELA,SumUp,16668076000120,Places,None,...,True,True,True,True,True,True,True,21378.0,227804,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072631,"MARIAFERREIRADASILVA|Araguaína TO, CLEBERFERRE...",564297,Araguaína,TO,<NA>,MULTCOSMETICOS,Ton,00000167491113,Places,None,...,True,False,True,False,False,True,True,1.0,227804,5
1072632,"GESSICASOARESDEALMEIDA|Araguaína TO, BARDOHEMA...",845138,Araguaína,TO,<NA>,BARDOHEMAN,Ton,06664036193,Ton,None,...,False,False,False,False,False,True,False,NaN,227804,2
1072633,"PEDROGOMESDASILVA|Araguaína TO, CREDIARIOSAOPE...",501724,Araguaína,TO,<NA>,CREDIARIOSAOPEDRO,Ton,,Places,None,...,True,True,True,True,True,True,True,214444.0,227804,3
1072634,"ROBERTOALVESPEREIRA|Araguaína TO, DECOREMOVEIS...",845139,Araguaína,TO,<NA>,DECOREMOVEIS,Ton,03463883171,Ton,None,...,False,False,False,False,False,True,False,NaN,227804,2


In [9]:
df[df['agrupamento_inspecao']==1943]['merchant_market_hierarchy_id'].unique()

<IntegerArray>
[     <NA>, 441200413, 417692441, 705450936, 543553940, 708741737, 286793418,
 791763576, 265071322, 624987117, 563643561, 746762014, 849126064, 402782003,
 735486402, 536457797, 415415619, 373672601, 715253292, 857715755, 685266288,
 614429530, 788881989, 585070466, 840931008, 761333292, 543258412, 747857723,
 600405552, 664384460, 578783809, 663489690, 503865932, 832735765, 824319714,
 571436529, 740438267, 375315963, 655836258, 822178082, 825823224, 764616813,
 457063262, 762150589, 789553492, 747155094, 651740206, 426462215, 844213218,
 777827423, 859163548, 746477184, 682128017, 760463576, 736605621, 844284317,
 796888244, 741579965, 706939709, 808246641, 740685771, 848922358, 824991185,
 808933375, 700963062, 826071702, 865946457, 852314114, 791037395, 699943413,
 778239299, 822463962, 664954996, 860666935, 747252864, 741130673, 782870638,
 865888239, 742723582, 651740088, 740441567, 751591374, 832581687, 753099211,
 789719995, 661841382, 778477556, 722558010, 7942

In [10]:
df.columns

Index(['group_id', 'group_id_index', 'nome_muni', 'uf',
       'merchant_market_hierarchy_id', 'nome_master', 'subs_asterisk',
       'documento_final', 'source_document', 'merchant_tax_id',
       'numero_inicio', 'cnpj', 'cpf', 'cpf_brasil', 'grouped_names',
       'qtd_cpfs', 'qtd_cnpjs', 'cod_muni', 'nome_ton', 'inicio',
       'agrupamento_nome_1', '_merge', 'mmhid_merge', 'muni', 'len_resultado',
       'document', 'agrupamento_inspecao', 'qtd_distinct_agrupamento_nome_1',
       'len_inspecao', 'has_mp', 'has_mp_subs', 'has_outros', 'has_pagseguro',
       'has_sumup', 'has_ton', 'has_not_ton'],
      dtype='object')

In [11]:
df['cod_muni']

0       2300200
1       2300200
2       2300200
3       2300200
4       2300200
         ...   
3981    2300200
3982    2300200
3983    2300200
3984    2300200
3985    2300200
Name: cod_muni, Length: 3986, dtype: Int64

In [157]:
df = pd.read_parquet('data/intermediary/part_11.parquet')

In [159]:
df = pd.read_parquet('data/tratados/part_11.parquet')

In [160]:
df

,group_id,group_id_index,nome_muni,uf,merchant_market_hierarchy_id,nome_master,subs_asterisk,documento_final,source_document,merchant_tax_id,...,has_mp,has_mp_subs,has_outros,has_pagseguro,has_sumup,has_ton,has_not_ton,qtd_mmhid_tax_id,grouped_tax_id,qtd_nomes
0,"SHOGUNTEAMOSASCOVD|Osasco SP, SHOGUN|Osasco SP",1,Osasco,SP,720191976,SHOGUNTEAMOSASCO,Outros,42071754000182,Places,42071754000182,...,True,True,True,True,False,True,True,1.0,1,3
1,FRANCISCORICARTESIL|Osasco SP,2,Osasco,SP,617184810,FRANCISCORICARTESIL,Outros,00000935966854,Places,00000935966854,...,False,False,True,False,False,False,True,1.0,2,1
2,JEANFERREIRA|Osasco SP,3,Osasco,SP,<NA>,JEANFERREIRA,MercadoPago,25124796000115,Places,25124796000115,...,True,True,True,True,False,True,True,0.0,728,1
3,EBURGER|Osasco SP,4,Osasco,SP,<NA>,EBURGER,MercadoPago,25124796000115,Places,25124796000115,...,True,True,True,True,False,True,True,0.0,728,1
4,"CLINICAODONTOLOGICA|Osasco SP, CLINICAODONTOLO...",5,Osasco,SP,538424095,CLINICAODONTOLOGICA,Outros,04129368000105,Places,04129368000105,...,True,False,True,False,False,False,True,3.0,728,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187092,"ALESSANDRAMAIADASILVA|Osasco SP, ALEMAIASTUDIO...",302853,Osasco,SP,<NA>,ALEMAIASTUDIO,Ton,35295383881,Ton,None,...,True,False,False,False,False,True,True,NaN,728,4
1187093,"JUSSARAJAQUELINESILVA|Osasco SP, JUSSARAESTETI...",1056542,Osasco,SP,<NA>,JUSSARAESTETICA,Ton,05368872682,Ton,None,...,False,False,False,False,False,True,False,NaN,728,2
1187094,"EDINILSONFERREIRADOCARMO|Osasco SP, EFMATERIAL...",1056543,Osasco,SP,<NA>,EFMATERIALPARACONSTRUCAO,Ton,28253626000129,Ton,None,...,False,False,False,False,False,True,False,NaN,728,2
1187095,"ELISANGELACOSTADUARTEDEALMEIDA|Osasco SP, CAFE...",16049,Osasco,SP,<NA>,CAFEDATIALI,Ton,09331882000142,Places,None,...,False,False,True,False,False,True,True,1.0,728,3


In [162]:
df['group_id_index'].nunique()

1056434

In [131]:
df = pd.read_parquet('data/nomes_agrupados/part_3534401.parquet')

In [133]:
df['resultado_names'].apply(lambda x: ','.join(x)).nunique()	

1058120

In [5]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os

import time

from projectutils import read_data 
import sys

In [6]:
#import sys, importlib
#importlib.reload(sys.modules['group_tam_id'])
from group_tam_id import (
    assign_group_ids,
    open_ton, load_to_gbq, create_doc_final, open_init_data,
    execute_with_context,
    init_group_id, 
    deal_merged_places, 
    deal_unmerged_places, 
    choose_prefered_document, 
    create_agrupamento_inspecao, 
    grouped_subs_asterisk, 
    final_ajustes)

In [7]:
import pandas as pd
import numpy as np
from datetime import datetime

def create_main_data(input_date: str):

    # Convert input date string to datetime object
    reference_date = np.datetime64(datetime.strptime(input_date, '%Y-%m-%d'))

    df = read_data('final_nomes')
    df = df[df['reference_month'] == reference_date]

    LEVEL_GROUP = ['cod_muni', 'reference_month']
    OUTROS = ['Outros', 'Outros_Pags', 'Outros_SumUp', 'Outros_Stone']

    print('Apenas mmhid de Outro')
    df['merchant_market_hierarchy_id'] = df['merchant_market_hierarchy_id'].where(df['subs_asterisk'].isin(OUTROS), None)

    df = df.drop_duplicates(['subs_asterisk', 'nome_master', 'merchant_market_hierarchy_id'] + LEVEL_GROUP)

    ton = open_ton(read_data, LEVEL_GROUP)
    ton = ton[ton['reference_month'] == reference_date]

    df = pd.concat([df, ton])
    df['nome_master'] = df['nome_master'].str.replace(r'[^A-Z]', '', regex=True)
    df['inicio'] = df['nome_master'].str[:6]

    df['cod_muni'] = df['cod_muni'].fillna(9999)
    df['nome_muni'] = df['nome_muni'].fillna('Desconhecido')
    df['uf'] = df['uf'].fillna('DE')

    # Save to parquet file with the input date in the filename
    df.to_parquet(f'data/main_data_{input_date}.parquet')


In [8]:
create_main_data('2024-03-31')

Apenas mmhid de Outro


In [ ]:
df = pd.read_parquet(